# Imports

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn

import tensorflow_hub as hub
import math
import bert

from tensorflow.keras.models import Model
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from bert.tokenization import bert_tokenization

FullTokenizer = bert_tokenization.FullTokenizer

In [2]:
np.set_printoptions(threshold=np.inf)

In [3]:
sampleDF = pd.read_csv('/Your/Path/to/hd2021.csv', encoding='unicode_escape')

# BERT Setup

In [8]:
#Model Setup
max_seq_length = 256  # Your choice here.

input_word_ids = tf.keras.layers.Input(shape = (max_seq_length,), dtype = tf.int32, name = "input_word_ids")
input_mask = tf.keras.layers.Input(shape = (max_seq_length,), dtype = tf.int32, name = "input_mask")
segment_ids = tf.keras.layers.Input(shape = (max_seq_length,), dtype = tf.int32, name = "segment_ids")

bert_layer = hub.KerasLayer('/Your/BERT/Model/Path/Here', trainable = True)

outputs = bert_layer({'input_mask': input_mask, 'input_type_ids': segment_ids, 'input_word_ids': input_word_ids})
pooled_output = outputs["pooled_output"]
sequence_output = outputs["sequence_output"]

model = Model(inputs = [input_word_ids, input_mask, segment_ids], outputs = [pooled_output, sequence_output])

In [8]:
#Manual processing of tokens
def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [9]:
#Create tokenizer that will create tokens
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [10]:
def documentEmbed(d):
    dtokens = tokenizer.tokenize(d)
    dtokens = ["[CLS]"] + dtokens + ["[SEP]"]

    input_ids_d = get_ids(dtokens, tokenizer, max_seq_length)
    input_masks_d = get_masks(dtokens, max_seq_length)
    input_segments_d = get_segments(dtokens, max_seq_length)

    input_ids_d=np.array(input_ids_d)
    input_masks_d = np.array(input_masks_d)
    input_segments_d = np.array(input_segments_d)

    pool_embs_d, all_embs_d = model.predict([input_ids_d.reshape(1,max_seq_length),input_masks_d.reshape(1,max_seq_length),input_segments_d.reshape(1,max_seq_length)])
    return pool_embs_d, all_embs_d

# hd2021 Setup

In [11]:
#Dropping numeric Columns
sampleDF = sampleDF[['INSTNM', 'IALIAS', 'ADDR', 'CITY', 'STABBR', 'CHFNM', 'CHFTITLE', 'COUNTYNM']]
sampleDF = sampleDF.fillna("")

In [12]:
#Removing special characters
alphabetFilterString = ' abcdefghijklmnopqrstuvwxyz'
sampleDF = sampleDF.applymap(lambda x: ''.join([y for y in x.lower() if y in alphabetFilterString]))

# Embedding Generation

### Context Window Size: 1

In [61]:
#Convert column segments of length 1 into sentence embeddings + labels
x1 = np.empty((0,768))

y1 = np.zeros((8,1000))
y1[0] = 1
for num in range(1000):
    nextX = sampleDF.INSTNM.sample(n=1, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x1 = np.append(x1, nextX, axis=0)
print("INSTNM DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[1]=1
y1 = np.concatenate((y1,yPlaceholder), axis=1)
sampleDF.IALIAS = sampleDF.IALIAS.replace(r'^\s*$', np.nan, regex=True)
for num in range(1000):
    nextX = sampleDF.IALIAS.dropna().sample(n=1, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x1 = np.append(x1, nextX, axis=0)
print("IALIAS DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[2]=1
ADDR = sampleDF.ADDR.replace(r'^\s*$', np.nan, regex=True)
y1 = np.concatenate((y1,yPlaceholder), axis=1)
for num in range(1000):
    nextX = ADDR.dropna().sample(n=1, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x1 = np.append(x1, nextX, axis=0)
print("ADDR DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[3]=1
y1 = np.concatenate((y1,yPlaceholder), axis=1)
for num in range(1000):
    nextX = sampleDF.CITY.sample(n=1, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x1 = np.append(x1, nextX, axis=0)
print("CITY DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[4]=1
y1 = np.concatenate((y1,yPlaceholder), axis=1)
for num in range(1000):
    nextX = sampleDF.STABBR.sample(n=1, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x1 = np.append(x1, nextX, axis=0)
print("STABBR DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[5]=1
CHFNM = sampleDF.CHFNM.replace(r'^\s*$', np.nan, regex=True)
y1 = np.concatenate((y1,yPlaceholder), axis=1)
for num in range(1000):
    nextX = CHFNM.dropna().sample(n=1, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x1 = np.append(x1, nextX, axis=0)
print("CHFNM DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[6]=1
CHFTITLE = sampleDF.CHFTITLE.replace(r'^\s*$', np.nan, regex=True)
y1 = np.concatenate((y1,yPlaceholder), axis=1)
for num in range(1000):
    nextX = CHFTITLE.dropna().sample(n=1, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x1 = np.append(x1, nextX, axis=0)
print("CHFTITLE DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[7]=1
COUNTYNM = sampleDF.COUNTYNM.replace(r'^\s*$', np.nan, regex=True)
y1 = np.concatenate((y1,yPlaceholder), axis=1)
for num in range(1000):
    nextX = COUNTYNM.dropna().sample(n=1, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x1 = np.append(x1, nextX, axis=0)
print("COUNTYNM DONE")

1/1 [==============================] - 0s 396ms/step


1/1 [==============================] - 0s 365ms/step


1/1 [==============================] - 0s 377ms/step


1/1 [==============================] - 0s 369ms/step
INSTNM DONE
1/1 [==============================] - 0s 364ms/step


1/1 [==============================] - 0s 368ms/step


...

1/1 [==============================] - 0s 356ms/step


1/1 [==============================] - 0s 355ms/step
CHFTITLE DONE
1/1 [==============================] - 0s 389ms/step


1/1 [==============================] - 0s 351ms/step


1/1 [==============================] - 0s 358ms/step


1/1 [==============================] - 0s 352ms/step
COUNTYNM DONE


In [62]:
print(x1.shape)
print(y1.shape)

(8000, 768)
(8, 8000)


### Context Window Size: 5

In [68]:
#Convert column segments of length 5 into sentence embeddings + labels
x5 = np.empty((0,768))

y5 = np.zeros((8,1000))
y5[0] = 1
for num in range(1000):
    nextX = sampleDF.INSTNM.sample(n=5, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x5 = np.append(x5, nextX, axis=0)
print("INSTNM DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[1]=1
y5 = np.concatenate((y5,yPlaceholder), axis=1)
sampleDF.IALIAS = sampleDF.IALIAS.replace(r'^\s*$', np.nan, regex=True)
for num in range(1000):
    nextX = sampleDF.IALIAS.dropna().sample(n=5, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x5 = np.append(x5, nextX, axis=0)
print("IALIAS DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[2]=1
ADDR = sampleDF.ADDR.replace(r'^\s*$', np.nan, regex=True)
y5 = np.concatenate((y5,yPlaceholder), axis=1)
for num in range(1000):
    nextX = ADDR.dropna().sample(n=5, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x5 = np.append(x5, nextX, axis=0)
print("ADDR DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[3]=1
y5 = np.concatenate((y5,yPlaceholder), axis=1)
for num in range(1000):
    nextX = sampleDF.CITY.sample(n=5, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x5 = np.append(x5, nextX, axis=0)
print("CITY DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[4]=1
y5 = np.concatenate((y5,yPlaceholder), axis=1)
for num in range(1000):
    nextX = sampleDF.STABBR.sample(n=5, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x5 = np.append(x5, nextX, axis=0)
print("STABBR DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[5]=1
CHFNM = sampleDF.CHFNM.replace(r'^\s*$', np.nan, regex=True)
y5 = np.concatenate((y5,yPlaceholder), axis=1)
for num in range(1000):
    nextX = CHFNM.dropna().sample(n=5, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x5 = np.append(x5, nextX, axis=0)
print("CHFNM DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[6]=1
CHFTITLE = sampleDF.CHFTITLE.replace(r'^\s*$', np.nan, regex=True)
y5 = np.concatenate((y5,yPlaceholder), axis=1)
for num in range(1000):
    nextX = CHFTITLE.dropna().sample(n=5, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x5 = np.append(x5, nextX, axis=0)
print("CHFTITLE DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[7]=1
COUNTYNM = sampleDF.COUNTYNM.replace(r'^\s*$', np.nan, regex=True)
y5 = np.concatenate((y5,yPlaceholder), axis=1)
for num in range(1000):
    nextX = COUNTYNM.dropna().sample(n=5, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x5 = np.append(x5, nextX, axis=0)
print("COUNTYNM DONE")

1/1 [==============================] - 0s 363ms/step


1/1 [==============================] - 0s 361ms/step


1/1 [==============================] - 0s 359ms/step


1/1 [==============================] - 0s 366ms/step
INSTNM DONE
1/1 [==============================] - 0s 364ms/step


1/1 [==============================] - 0s 359ms/step


...

1/1 [==============================] - 0s 355ms/step


1/1 [==============================] - 0s 357ms/step
CHFTITLE DONE
1/1 [==============================] - 0s 354ms/step


1/1 [==============================] - 0s 349ms/step


1/1 [==============================] - 0s 346ms/step


1/1 [==============================] - 0s 392ms/step
COUNTYNM DONE


In [69]:
print(x5.shape)
print(y5.shape)

(8000, 768)
(8, 8000)


### Context Window Size: 10

In [19]:
#Convert column segments of length 10 into sentence embeddings + labels
x10 = np.empty((0,768))

y10 = np.zeros((8,1000))
y10[0] = 1
for num in range(1000):
    nextX = sampleDF.INSTNM.sample(n=10, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x10 = np.append(x10, nextX, axis=0)
print("INSTNM DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[1]=1
y10 = np.concatenate((y10,yPlaceholder), axis=1)
sampleDF.IALIAS = sampleDF.IALIAS.replace(r'^\s*$', np.nan, regex=True)
for num in range(1000):
    nextX = sampleDF.IALIAS.dropna().sample(n=10, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x10 = np.append(x10, nextX, axis=0)
print("IALIAS DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[2]=1
ADDR = sampleDF.ADDR.replace(r'^\s*$', np.nan, regex=True)
y10 = np.concatenate((y10,yPlaceholder), axis=1)
for num in range(1000):
    nextX = ADDR.dropna().sample(n=10, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x10 = np.append(x10, nextX, axis=0)
print("ADDR DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[3]=1
y10 = np.concatenate((y10,yPlaceholder), axis=1)
for num in range(1000):
    nextX = sampleDF.CITY.sample(n=10, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x10 = np.append(x10, nextX, axis=0)
print("CITY DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[4]=1
y10 = np.concatenate((y10,yPlaceholder), axis=1)
for num in range(1000):
    nextX = sampleDF.STABBR.sample(n=10, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x10 = np.append(x10, nextX, axis=0)
print("STABBR DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[5]=1
CHFNM = sampleDF.CHFNM.replace(r'^\s*$', np.nan, regex=True)
y10 = np.concatenate((y10,yPlaceholder), axis=1)
for num in range(1000):
    nextX = CHFNM.dropna().sample(n=10, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x10 = np.append(x10, nextX, axis=0)
print("CHFNM DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[6]=1
CHFTITLE = sampleDF.CHFTITLE.replace(r'^\s*$', np.nan, regex=True)
y10 = np.concatenate((y10,yPlaceholder), axis=1)
for num in range(1000):
    nextX = CHFTITLE.dropna().sample(n=10, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x10 = np.append(x10, nextX, axis=0)
print("CHFTITLE DONE")

yPlaceholder= np.zeros((8,1000))
yPlaceholder[7]=1
COUNTYNM = sampleDF.COUNTYNM.replace(r'^\s*$', np.nan, regex=True)
y10 = np.concatenate((y10,yPlaceholder), axis=1)
for num in range(1000):
    nextX = COUNTYNM.dropna().sample(n=10, random_state = num)
    nextX = documentEmbed(' '.join(nextX))[1]
    nextX = np.sum(nextX, axis=1)
    x10 = np.append(x10, nextX, axis=0)
print("COUNTYNM DONE")

1/1 [==============================] - 0s 369ms/step


1/1 [==============================] - 0s 425ms/step


1/1 [==============================] - 0s 371ms/step


1/1 [==============================] - 0s 366ms/step
INSTNM DONE
1/1 [==============================] - 0s 372ms/step


1/1 [==============================] - 0s 366ms/step


...

1/1 [==============================] - 0s 352ms/step


1/1 [==============================] - 0s 353ms/step
CHFTITLE DONE
1/1 [==============================] - 0s 359ms/step


1/1 [==============================] - 0s 354ms/step


1/1 [==============================] - 0s 349ms/step


1/1 [==============================] - 0s 349ms/step
COUNTYNM DONE


In [20]:
print(x10.shape)
print(y10.shape)

(8000, 768)
(8, 8000)


# Training Models

### Context Window Size: 1

In [82]:
input_shape = x1.shape

model1 = models.Sequential()
model1.add(layers.Dense(1000, input_shape = input_shape[1:], activation='relu'))
model1.add(layers.Dense(8, activation = 'softmax'))

In [83]:
xTrain1, xTest1, yTrain1, yTest1 = train_test_split(x1, y1.transpose(), test_size=0.55, random_state=42)
print(xTrain1.shape)

(3600, 768)


In [84]:
model1.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
model1.fit(xTrain1, yTrain1, epochs=10)

Epoch 1/10
113/113 [==============================] - 0s 2ms/step - loss: 3.5501 - accuracy: 0.8267
Epoch 2/10
113/113 [==============================] - 0s 2ms/step - loss: 0.8866 - accuracy: 0.9136
Epoch 3/10
113/113 [==============================] - 0s 2ms/step - loss: 0.4144 - accuracy: 0.9356
Epoch 4/10
113/113 [==============================] - 0s 2ms/step - loss: 0.2542 - accuracy: 0.9511
Epoch 5/10
113/113 [==============================] - 0s 2ms/step - loss: 0.1977 - accuracy: 0.9622
Epoch 6/10
113/113 [==============================] - 0s 2ms/step - loss: 0.1929 - accuracy: 0.9575
Epoch 7/10
113/113 [==============================] - 0s 2ms/step - loss: 0.1096 - accuracy: 0.9692
Epoch 8/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0674 - accuracy: 0.9794
Epoch 9/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0612 - accuracy: 0.9803
Epoch 10/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0498 - accuracy: 0.9819

In [85]:
test_loss, test_acc = model1.evaluate(xTest1, yTest1, verbose=2)
predictionSeries = np.argmax(model1.predict(xTest1), axis=-1)
print(predictionSeries)
print(predictionSeries.shape)

138/138 - 0s - loss: 0.4862 - accuracy: 0.9170 - 148ms/epoch - 1ms/step
138/138 [==============================] - 0s 655us/step
[2 2 1 1 4 2 1 1 4 3 1 2 1 4 4 4 7 4 2 2 1 5 6 7 0 3 7 3 0 3 0 7 6 0 6 0 5
 4 0 1 2 2 5 2 3 5 1 6 4 6 2 4 4 6 1 7 2 6 0 0 5 2 0 2 4 7 6 5 0 3 3 0 3 4
 7 1 1 5 5 3 6 3 0 2 1 6 5 4 2 0 3 4 0 6 0 0 0 6 6 2 0 1 2 7 6 4 3 4 4 0 0
 5 2 0 4 4 7 1 0 5 2 6 6 1 5 1 0 7 4 4 5 0 4 6 3 5 0 6 3 6 6 6 2 7 1 2 0 0
 6 3 2 1 3 1 3 2 6 3 1 0 6 7 0 1 4 1 1 5 7 5 4 1 0 6 1 6 7 4 2 5 5 5 0 7 4
 5 3 1 6 1 0 3 0 1 7 2 4 1 3 7 4 0 5 0 5 3 4 7 7 0 2 3 6 3 1 0 4 3 0 0 1 1
 4 7 6 1 7 6 0 5 0 7 2 0 5 4 3 0 4 0 7 5 0 7 1 2 2 3 7 4 0 3 6 2 0 1 2 0 7
 7 2 2 2 0 7 2 7 2 0 6 7 5 2 2 5 3 3 0 6 2 1 0 6 1 7 0 4 6 0 0 7 2 1 7 4 4
 3 6 0 1 2 0 2 2 3 6 0 6 0 0 3 4 3 7 5 2 5 2 0 1 1 1 7 3 6 2 7 6 7 7 7 1 6
 4 5 1 2 1 5 7 6 3 2 4 6 4 0 7 7 5 4 4 7 0 5 2 4 6 6 7 5 3 4 7 2 0 0 4 4 7
 4 4 3 6 5 3 1 2 1 7 1 1 5 0 6 7 6 5 4 1 6 4 5 0 2 6 1 6 7 3 0 0 0 7 1 0 6
 2 6 0 2 0 7 6 1 7 4 2 3 7 0 4 1 1 7 3 0 6 2 7

### Context Window Size: 5

In [78]:
input_shape = x5.shape

model5= models.Sequential()
model5.add(layers.Dense(1000, input_shape = input_shape[1:], activation='relu'))
model5.add(layers.Dense(8, activation = 'softmax'))

In [79]:
xTrain5, xTest5, yTrain5, yTest5 = train_test_split(x5, y5.transpose(), test_size=0.55, random_state=42)
print(xTrain5.shape)

(3600, 768)


In [80]:
model5.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
model5.fit(xTrain5, yTrain5, epochs=10)

Epoch 1/10
113/113 [==============================] - 0s 2ms/step - loss: 1.2349 - accuracy: 0.9422
Epoch 2/10
113/113 [==============================] - 0s 2ms/step - loss: 0.2240 - accuracy: 0.9847
Epoch 3/10
113/113 [==============================] - 0s 2ms/step - loss: 0.2068 - accuracy: 0.9842
Epoch 4/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0979 - accuracy: 0.9906
Epoch 5/10
113/113 [==============================] - 0s 2ms/step - loss: 0.1471 - accuracy: 0.9908
Epoch 6/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0980 - accuracy: 0.9917
Epoch 7/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0510 - accuracy: 0.9956
Epoch 8/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0324 - accuracy: 0.9967
Epoch 9/10
113/113 [==============================] - 0s 2ms/step - loss: 0.1476 - accuracy: 0.9911
Epoch 10/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0188 - accuracy: 0.9981

In [81]:
test_loss, test_acc = model5.evaluate(xTest5, yTest5, verbose=2)
predictionSeries = np.argmax(model5.predict(xTest5), axis=-1)
print(predictionSeries)
print(predictionSeries.shape)

138/138 - 0s - loss: 0.1215 - accuracy: 0.9880 - 149ms/epoch - 1ms/step
138/138 [==============================] - 0s 651us/step
[2 2 1 3 4 2 1 1 4 3 1 2 2 4 4 4 7 4 2 2 3 5 6 7 0 3 7 3 0 3 0 7 6 1 6 1 5
 4 0 1 2 2 5 2 3 5 3 6 4 6 2 4 4 6 1 7 2 6 7 0 5 2 0 2 4 7 6 5 1 3 3 0 3 4
 7 1 1 5 2 3 6 3 0 2 1 6 5 4 2 0 3 4 0 6 1 0 1 6 6 2 1 1 2 7 6 4 3 1 4 1 1
 5 2 0 4 4 7 1 0 5 2 6 6 1 5 1 1 7 4 4 5 0 4 6 3 5 1 6 3 6 6 6 2 7 1 2 0 2
 6 3 2 3 3 3 3 2 6 6 0 0 6 7 0 1 4 1 5 5 7 5 4 1 0 6 1 6 7 4 2 5 5 5 0 7 4
 5 3 1 6 1 1 3 0 1 3 2 4 1 3 7 4 1 5 0 5 3 4 7 7 0 2 3 6 3 0 0 4 3 0 0 1 1
 4 7 6 1 7 6 1 5 0 7 2 0 5 4 3 0 4 1 7 5 0 7 1 1 2 3 7 4 0 3 6 2 0 6 2 0 7
 7 2 2 2 0 7 2 7 2 0 6 7 5 2 2 5 3 3 0 6 2 2 0 6 1 7 1 4 6 1 0 7 2 1 7 4 4
 3 6 1 0 1 0 2 2 3 6 0 6 0 0 3 4 3 7 0 3 5 2 0 0 1 1 7 3 6 2 7 6 7 7 7 1 6
 4 5 1 2 1 5 7 6 3 2 4 6 4 0 7 7 5 4 4 7 0 5 2 4 6 6 7 5 3 4 7 2 1 0 4 4 7
 4 4 3 6 5 5 1 2 1 7 1 1 5 1 6 7 6 5 4 1 6 4 5 0 2 6 1 6 7 3 0 0 0 7 1 0 6
 2 6 0 2 0 7 6 1 7 4 2 3 7 0 4 1 1 7 3 0 6 2 7

### Context Window Size: 10

In [21]:
input_shape = x10.shape

model10 = models.Sequential()
model10.add(layers.Dense(1000, input_shape = input_shape[1:], activation='relu'))
model10.add(layers.Dense(8, activation = 'softmax'))

In [22]:
xTrain10, xTest10, yTrain10, yTest10 = train_test_split(x10, y10.transpose(), test_size=0.55, random_state=42)
print(xTrain10.shape)

(3600, 768)


In [23]:
model10.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
model10.fit(xTrain10, yTrain10, epochs=10)

Epoch 1/10
113/113 [==============================] - 0s 2ms/step - loss: 0.8391 - accuracy: 0.9586
Epoch 2/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0741 - accuracy: 0.9942
Epoch 3/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0401 - accuracy: 0.9972
Epoch 4/10
113/113 [==============================] - 0s 2ms/step - loss: 0.1190 - accuracy: 0.9931
Epoch 5/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0380 - accuracy: 0.9983
Epoch 6/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0458 - accuracy: 0.9964
Epoch 7/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0787 - accuracy: 0.9961
Epoch 8/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0228 - accuracy: 0.9983
Epoch 9/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0345 - accuracy: 0.9978
Epoch 10/10
113/113 [==============================] - 0s 2ms/step - loss: 0.0235 - accuracy: 0.9989

In [24]:
test_loss, test_acc = model10.evaluate(xTest10, yTest10, verbose=2)
predictionSeries = np.argmax(model10.predict(xTest10), axis=-1)
print(predictionSeries)
print(predictionSeries.shape)

138/138 - 0s - loss: 0.0474 - accuracy: 0.9975 - 162ms/epoch - 1ms/step
138/138 [==============================] - 0s 671us/step
[2 2 1 3 4 2 1 1 4 3 1 2 2 4 4 4 7 4 2 2 3 5 6 7 0 3 7 3 0 3 0 7 6 1 6 1 5
 4 0 1 2 2 5 2 3 5 1 6 4 6 2 4 4 6 1 7 2 6 7 0 5 2 0 2 4 7 6 5 1 3 3 0 3 4
 7 1 1 5 2 3 6 3 0 2 1 6 5 4 2 0 3 4 0 6 1 0 1 6 6 2 1 1 2 7 6 4 3 1 4 1 1
 5 2 0 4 4 7 1 0 5 2 6 6 1 5 1 1 7 4 4 5 0 4 6 3 5 1 6 3 6 6 6 2 7 1 2 0 2
 6 3 2 3 3 3 3 2 6 6 0 0 6 7 0 1 4 1 5 5 7 5 4 1 0 6 1 6 7 4 2 5 5 5 0 7 4
 5 3 1 6 1 1 3 0 1 3 2 4 1 3 7 4 1 5 0 5 3 4 7 7 0 2 3 6 3 0 0 4 3 0 0 1 1
 4 7 6 1 7 6 1 5 0 7 2 0 5 4 3 0 4 1 7 5 0 7 1 1 2 3 7 4 0 3 6 2 0 6 2 0 7
 7 2 2 2 0 7 2 7 2 0 6 7 5 2 2 5 3 3 0 6 2 2 0 6 1 7 1 4 6 1 0 7 2 1 7 4 4
 3 6 0 1 1 0 2 2 3 6 0 6 0 0 3 4 3 7 0 3 5 2 0 0 1 1 7 3 6 2 7 6 7 7 7 1 6
 4 5 1 2 1 5 7 6 3 2 4 6 4 1 7 7 5 4 4 7 0 5 2 4 6 6 7 5 3 4 7 2 1 0 4 4 7
 4 4 3 6 5 5 1 2 1 7 1 1 5 1 6 7 6 5 4 1 6 4 5 0 2 6 1 6 7 3 0 0 0 7 1 0 6
 2 6 0 2 0 7 6 1 7 4 2 3 7 0 4 1 1 7 3 0 6 2 7